In [ ]:
"""
IMPORTS
"""

import pandas as pd
import numpy as np
import onnxruntime as rt
import dice_ml
from dice_ml.utils import helpers 

from utils.dataloader import DataLoader
from models.modelloader import ModelLoader
from utils.constants import protected_attributes, group_proxies


from tests.metrics import MetricsTester
from tests.mututation import MutationTester
from tests.metamorphic import MetamorphicTester
from tests.shapley import ShapleyTester

In [ ]:
"""
    INTERESTING LINKS:
        - https://dl.acm.org/doi/pdf/10.1145/3338906.3338937
"""

In [1]:
"""
SET MODEL(S)
"""


model_loader = ModelLoader(type='good')

In [ ]:
"""
LOAD DATASET
"""

dataloader = DataLoader()
X_train, y_train = dataloader.load_split('train')
print(f'{X_train.shape}, {y_train.shape}')
X_test, y_test = dataloader.load_split('test')
print(f'{X_test.shape}, {y_test.shape}')
X_data, y_data = dataloader.load_split('full')
print(f'{X_data.shape}, {y_data.shape}')

In [ ]:
"""
    TEST MODEL ON DATASET
"""

model_loader.fit(X_train, y_train)
y_pred = model_loader.predict()

In [ ]:
"""
FUNCTIONAL TESTING
"""

metrics = MetricsTester(protected_variables=protected_attributes, group_metrics=group_proxies)


In [ ]:
metrics.get_metrics_summary(X_test, y_true=y_test, y_pred=y_pred)

In [ ]:
"""
MUTATION TESTING
"""

params = {
    'data_shuffler': 1000,
    'data_remover': 0.25,
    'data_repetition': 1000, 
    'label_error': 1000,
    'feature_remover': 31,
}


mutator = MutationTester(y_pred_baseline=y_pred, df_train=X_train, df_test=X_test)
mutation_score = mutator.test_mutants(params)
print(f"Mutation score: {mutation_score}")

In [ ]:
"""
AUC
"""


In [ ]:
"""
SHAP VALUES
"""

shapley_tester = ShapleyTester()
shapley_explainer = ShapleyTester.get_model_explainer(model=model_loader.model.model)
shapley_tester.plot_shapley_values(shapley_explainer, X_test.iloc[0])


In [ ]:
"""
COUNTERFACTUALS
"""

# Model

data = dice_ml.Data(dataframe=X_data, 
                   continuous_features=['persoon_leeftijd_bij_onderzoek'],
                   outcome_name='checked')

rf_dice = dice_ml.Model(model=model_loader.model.model, 
                        # There exist backends for tf, torch, ...
                        backend="sklearn")

explainer = dice_ml.Dice(data, 
                         rf_dice, 
                         # Random sampling, genetic algorithm, kd-tree,...
                         method="random")


In [ ]:
original_input = X_test[0:4]
cf_examples = explainer.generate_counterfactuals(original_input,
                                           total_CFs=3,
                                           desired_class="opposite")
cf_examples.visualize_as_dataframe(show_only_changes=True)